In [1]:
!pip install qiskit
!pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 74.6 MB/s eta 0:00:00


**TASK 1 :CUSTOM ORACLE DESIGN**

In [2]:
!pip install qiskit qiskit-aer -q

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

def deutsch_oracle_xxor1(qc):
    # Implements |x>|y> → |x>|y XOR (x XOR 1)>
    qc.x(1)      # Adds the +1 term
    qc.cx(0, 1)  # Adds XOR with x

# Build circuit
qc = QuantumCircuit(2, 1)
qc.x(1)                 # prepare output |1>
qc.h([0,1])             # Hadamard on both
qc.barrier()
deutsch_oracle_xxor1(qc)
qc.barrier()
qc.h(0)
qc.measure(0, 0)

print(qc.draw(output="text"))

# Simulate
sim = AerSimulator()
result = sim.run(transpile(qc, sim)).result()
counts = result.get_counts()
print("\nMeasurement:", counts)
print("Interpretation: 0 → constant, 1 → balanced")

     ┌───┐      ░            ░ ┌───┐┌─┐
q_0: ┤ H ├──────░────────■───░─┤ H ├┤M├
     ├───┤┌───┐ ░ ┌───┐┌─┴─┐ ░ └───┘└╥┘
q_1: ┤ X ├┤ H ├─░─┤ X ├┤ X ├─░───────╫─
     └───┘└───┘ ░ └───┘└───┘ ░       ║ 
c: 1/════════════════════════════════╩═
                                     0 

Measurement: {'1': 1024}
Interpretation: 0 → constant, 1 → balanced


**TASK 2 :BLOCH SPERE VISUALIZATION**

In [3]:
%matplotlib inline

from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector
import matplotlib.pyplot as plt

# After Hadamard
qc_h = QuantumCircuit(2)
qc_h.x(1)
qc_h.h([0,1])
sv_h = Statevector.from_instruction(qc_h)
print("Statevector after Hadamard:\n", sv_h)
plot_bloch_multivector(sv_h)
plt.show()

# After Oracle f(x)=x XOR 1
qc_o = QuantumCircuit(2)
qc_o.x(1)
qc_o.h([0,1])
qc_o.x(1)
qc_o.cx(0,1)
sv_o = Statevector.from_instruction(qc_o)
print("Statevector after Oracle:\n", sv_o)
plot_bloch_multivector(sv_o)
plt.show()


Statevector after Hadamard:
 Statevector([ 0.5+0.j,  0.5+0.j, -0.5+0.j, -0.5+0.j],
            dims=(2, 2))
Statevector after Oracle:
 Statevector([-0.5+0.j,  0.5+0.j,  0.5+0.j, -0.5+0.j],
            dims=(2, 2))


**TASK 3 :DEUTSCH-JOZSA EXTENSION**

In [4]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

def oracle_constant_0(qc, n):
    pass  # f(x) = 0

def oracle_balanced_xor(qc, n):
    # f(x0,x1)=x0 XOR x1
    qc.cx(0, n)
    qc.cx(1, n)

def deutsch_jozsa(n, oracle_fn):
    qc = QuantumCircuit(n+1, n)
    qc.x(n)
    qc.h(range(n+1))
    qc.barrier()
    oracle_fn(qc, n)
    qc.barrier()
    qc.h(range(n))
    qc.measure(range(n), range(n))
    return qc

n = 2
sim = AerSimulator()

# Balanced oracle
qc_balanced = deutsch_jozsa(n, oracle_balanced_xor)
res_bal = sim.run(transpile(qc_balanced, sim)).result().get_counts()
print("Balanced oracle counts:", res_bal)

# Constant oracle
qc_const = deutsch_jozsa(n, oracle_constant_0)
res_const = sim.run(transpile(qc_const, sim)).result().get_counts()
print("Constant oracle counts:", res_const)

Balanced oracle counts: {'11': 1024}
Constant oracle counts: {'00': 1024}


**TASK 4 : NOISE IMPACT ANALYSIS**

In [5]:
!pip install qiskit qiskit-aer -q

# Add quantum noise and analyze its impact
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error

# Build a simple noise model
noise_model = NoiseModel()
p1, p2 = 0.001, 0.01  # noise probabilities
noise_model.add_all_qubit_quantum_error(depolarizing_error(p1, 1), ['x','h','sx','u1','u2','u3'])
noise_model.add_all_qubit_quantum_error(depolarizing_error(p2, 2), ['cx'])

# Reuse previous qc_balanced (from Task 3)
sim_noisy = AerSimulator(noise_model=noise_model)
job = sim_noisy.run(transpile(qc_balanced, sim_noisy), shots=4096)
counts_noisy = job.result().get_counts()

print("Noisy measurement results:\n", counts_noisy)


Noisy measurement results:
 {'10': 8, '00': 8, '01': 32, '11': 4048}


**TASK 5 : COMPARE CLASSICAL VS QUIBIT QUERIES**

In [6]:
def f_1bit_xxor1(x): return x ^ 1

def classical_det_check_1bit(eval_fn):
    qcount = 0
    r0 = eval_fn(0); qcount += 1
    r1 = eval_fn(1); qcount += 1
    result = "constant" if r0 == r1 else "balanced"
    return result, qcount

res_classical, queries = classical_det_check_1bit(f_1bit_xxor1)
print("Classical deterministic:", res_classical, "| queries used:", queries)
print("Quantum Deutsch uses only 1 query.\n")

def f_2bit_parity(x):
    b0 = x & 1
    b1 = (x >> 1) & 1
    return b0 ^ b1

def classical_worst_case_queries(n):
    return (2**(n-1)) + 1

print("Classical worst-case queries for n=2:", classical_worst_case_queries(2))
print("Quantum Deutsch–Jozsa queries:", 1)


Classical deterministic: balanced | queries used: 2
Quantum Deutsch uses only 1 query.

Classical worst-case queries for n=2: 3
Quantum Deutsch–Jozsa queries: 1
